# Assignment 2: Milestone I Natural Language Processing
## Task 1. Basic Text Pre-processing
#### Student Name: Hongjie Xu
#### Student ID: s3880497

Date: 02/10/2021

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used:
* pandas
* re
* numpy
* os
* nltk
* sklearn
* itertools

## Introduction

Pre-processing text is the first step of text mining/ natural language task. Analyzing the text file structure, recognizing the potential pattern in the text file could help us find an approprate way to extract useful information from text. 


## Importing libraries 

In [1]:
# Code to import libraries as you need in this assessment
import re
import nltk
import os
import numpy as np
from nltk.tokenize import RegexpTokenizer
from sklearn.datasets import load_files  
from nltk.probability import *
from itertools import chain
from nltk.util import ngrams
from nltk import FreqDist

### 1.1 Examining and loading data
- Examine the data folder
    1. Eight Categories(Folders)
    2. 55449 txt Files
    3. Each txt file has either 3 lines or 4 lines. 
        * Each line has one attribute
        * Files with 3 lines have no `Company` information
        
- Using **Dictionary** Python data type to save each txt file (Key is attribute name, Value is relevant information), then saving all dictionaries to a list.

- Inter


In [2]:
# Using OS package to investigate the number of folders
print("Number of job categories is ", len(os.listdir('./data/')))
print("-----------------------------------------------------")
# Show job categories
for i in os.listdir('./data/'):
    print(i)
# Show number of file in each job category
print("-----------------------------------------------------")
for folder in os.listdir('./data/'):
    print(len(os.listdir(('./data/' + folder))), " in ", folder)
print("-----------------------------------------------------")

total_file_number=0
for folder in os.listdir('./data/'):
    total_file_number+= len(os.listdir(('./data/' + folder)))
print('Total document number is ', total_file_number)

Number of job categories is  8
-----------------------------------------------------
Accounting_Finance
Engineering
Healthcare_Nursing
Hospitality_Catering
IT
PR_Advertising_Marketing
Sales
Teaching
-----------------------------------------------------
7407  in  Accounting_Finance
8210  in  Engineering
8808  in  Healthcare_Nursing
4788  in  Hospitality_Catering
14353  in  IT
2755  in  PR_Advertising_Marketing
5349  in  Sales
3779  in  Teaching
-----------------------------------------------------
Total document number is  55449


In [3]:
# Check file extension & txt file
txt_num=0
for folder in os.listdir('./data/'):
    for document in os.listdir('./data/'+folder):
        if document.endswith('.txt'):
            txt_num+=1
print(txt_num)

55449


In [4]:
# number of lines in txt files
type_file={'3 lines':0,
           '4 lines':0,
           '2 lines':0,
           'greater than 4':0}
for folder in os.listdir('./data/'):
    for document in os.listdir('./data/'+folder):
        with open('./data/'+folder+'/'+document, 'rb') as f:
            line_num = len(f.readlines())
            if line_num == 3:
                type_file['3 lines']+=1
            elif line_num==4:
                type_file['4 lines']+=1

print(type_file)
print('Total document number is: ', sum(type_file.values()))

{'3 lines': 5388, '4 lines': 50061, '2 lines': 0, 'greater than 4': 0}
Total document number is:  55449


For now, all txt file are invesitated, they all have 3 or 4 lines. Total file number is 55449.

Lets look at the different between 3 lines file and 4 lines file.

In [5]:
# finding the reason why txt files has different numbers of lines
with open('./data/Accounting_Finance/Job_22564.txt', 'rb') as f:
    for i in f.readlines():
        print(i)
print('-----------------------------------------------------') 
with open('./data/Accounting_Finance/Job_22565.txt', 'rb') as f:
    for i in f.readlines():
        print(i)

b'Title: Trainee Mortgage Advisor  East Midlands\n'
b'Webindex: 19047429\n'
b'Company: Brite Recruitment\n'
b'Description: Are you a successful, results driven person? Are you looking to start a career in Financial Services? Are you looking for a fast paced, dynamic working environment? Our Client is proud to be an independent estate agent with an unbeatable local knowledge and reputation for excellent service. They have a mortgage advisor within each of their branches to accommodate their customer s needs. As a Trainee Mortgage Advisor, you will work closely with your team to learn and develop whilst achieving shared goals. In addition to this; You will have the determination to succeed in the Financial Services market. You will be capable of working effectively under pressure and in a target orientated environment. You will have the ability to successfully achieve your CeMap qualifications. You will be able to evaluate business at every opportunity. The company will offer a fantastic

Here we found, 3 lines file has no company information!

Now, its time to load data. 

In [6]:
# Using function provided by sklearn to load all files under specific directory
job_data = load_files("./data/")

In [7]:
# check up the targetname, which is actually each folder's name
job_data['target_names']

['Accounting_Finance',
 'Engineering',
 'Healthcare_Nursing',
 'Hospitality_Catering',
 'IT',
 'PR_Advertising_Marketing',
 'Sales',
 'Teaching']

In [8]:
# job id can be acquired using 'filename'
job_data['filenames'][0]

'./data/Engineering\\Job_14624.txt'

In [9]:
# Inspect the data, it is encoded with byte type, we need to decode it 
job_data.data[0]

b"Title: Plant Engineer\nWebindex: 62119057\nCompany: W5 Recruitment\nDescription: Our client has established itself as a leading manufacturer and supplier of quality water treatment plants, ranging from basic water softeners and reverse osmosis equipment to customer specified complex water treatment solutions. The company are able to meet their clients' requirements through flexibility in tailoring their product to their needs and budgets. Due to expansion and an increased workload they are seeking to recruit a Planet Engineer to cover accounts along the M4 Corridor Responsibilities will include conducting the routine sampling and analysis of water systems, interpreting results, maintenance and the installation of chemical dosing systems. Servicing accounts within both the industrial and commercial industries the successful candidate will complete all work in accordance to the approved code of practice. The ideal applicant for this position will have a minimum of two years relevant in

In [10]:
# assignment data and target to a new variable
job_description, job_category = job_data.data, job_data.target

In [11]:
# try to decode using utf-8, split by <'\n'>
job_description[0].decode('utf-8').split('\n')

['Title: Plant Engineer',
 'Webindex: 62119057',
 'Company: W5 Recruitment',
 "Description: Our client has established itself as a leading manufacturer and supplier of quality water treatment plants, ranging from basic water softeners and reverse osmosis equipment to customer specified complex water treatment solutions. The company are able to meet their clients' requirements through flexibility in tailoring their product to their needs and budgets. Due to expansion and an increased workload they are seeking to recruit a Planet Engineer to cover accounts along the M4 Corridor Responsibilities will include conducting the routine sampling and analysis of water systems, interpreting results, maintenance and the installation of chemical dosing systems. Servicing accounts within both the industrial and commercial industries the successful candidate will complete all work in accordance to the approved code of practice. The ideal applicant for this position will have a minimum of two years re

In [12]:
# check numbers of file again, exactly same as we found before
len(job_data.filenames)

55449

Six main elements in these files:
- `Filename`
- `Category`
- `Title`
- `Webindex`
- `Company`
- `Description`

We could save all these information using dictionary

In [13]:
# lets save all theses information to a list of dictionaries

job_list=[]
for idx,job in enumerate(job_description):
    job_dict={}
    job_dict['Filename']=job_data.filenames[idx][-9:-4] # saving job id
    job_dict['Category']=job_data.filenames[idx][job_data.filenames[2].find('data/')+5: job_data.filenames[idx].find('\\')] # saving category, which is folder name
    for i in job.decode('utf-8').split('\n'):
        if i.startswith('Title: '):
            job_dict['Title']=i.split(': ')[1] # saving title
        if i.startswith('Webindex: '):
            job_dict['Webindex']=i.split(': ')[1] # saving Webindex
        if i.startswith('Company: '):
            job_dict['Company']=i.split(': ')[1] # saving Company
        if i.startswith('Description: '):
            job_dict['Description']=i[i.find('Description: ') + len('Description: '):] # saving Description
    job_list.append(job_dict)

In [14]:
# inspect one of dictionary
job_list[0]

{'Filename': '14624',
 'Category': 'Engineering',
 'Title': 'Plant Engineer',
 'Webindex': '62119057',
 'Company': 'W5 Recruitment',
 'Description': "Our client has established itself as a leading manufacturer and supplier of quality water treatment plants, ranging from basic water softeners and reverse osmosis equipment to customer specified complex water treatment solutions. The company are able to meet their clients' requirements through flexibility in tailoring their product to their needs and budgets. Due to expansion and an increased workload they are seeking to recruit a Planet Engineer to cover accounts along the M4 Corridor Responsibilities will include conducting the routine sampling and analysis of water systems, interpreting results, maintenance and the installation of chemical dosing systems. Servicing accounts within both the industrial and commercial industries the successful candidate will complete all work in accordance to the approved code of practice. The ideal appli

### 1.2 Pre-processing data
Perform the required text pre-processing steps.

In [15]:
# get job description list from dictionary we created

description = []
for job in job_list:
    description.append(job['Description'])

description

["Our client has established itself as a leading manufacturer and supplier of quality water treatment plants, ranging from basic water softeners and reverse osmosis equipment to customer specified complex water treatment solutions. The company are able to meet their clients' requirements through flexibility in tailoring their product to their needs and budgets. Due to expansion and an increased workload they are seeking to recruit a Planet Engineer to cover accounts along the M4 Corridor Responsibilities will include conducting the routine sampling and analysis of water systems, interpreting results, maintenance and the installation of chemical dosing systems. Servicing accounts within both the industrial and commercial industries the successful candidate will complete all work in accordance to the approved code of practice. The ideal applicant for this position will have a minimum of two years relevant industry experience and will have knowledge of reverse osmosis, water softeners, wa

In [16]:
# Step 2, tokenize all description, using provided regular expression

pattern = r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?"

description_tokenized = []
for i in description:
    tokenizer = RegexpTokenizer(pattern) 
    description_tokenized.append(tokenizer.tokenize(i))

# inspect the result
description_tokenized[0]

['Our',
 'client',
 'has',
 'established',
 'itself',
 'as',
 'a',
 'leading',
 'manufacturer',
 'and',
 'supplier',
 'of',
 'quality',
 'water',
 'treatment',
 'plants',
 'ranging',
 'from',
 'basic',
 'water',
 'softeners',
 'and',
 'reverse',
 'osmosis',
 'equipment',
 'to',
 'customer',
 'specified',
 'complex',
 'water',
 'treatment',
 'solutions',
 'The',
 'company',
 'are',
 'able',
 'to',
 'meet',
 'their',
 'clients',
 'requirements',
 'through',
 'flexibility',
 'in',
 'tailoring',
 'their',
 'product',
 'to',
 'their',
 'needs',
 'and',
 'budgets',
 'Due',
 'to',
 'expansion',
 'and',
 'an',
 'increased',
 'workload',
 'they',
 'are',
 'seeking',
 'to',
 'recruit',
 'a',
 'Planet',
 'Engineer',
 'to',
 'cover',
 'accounts',
 'along',
 'the',
 'M',
 'Corridor',
 'Responsibilities',
 'will',
 'include',
 'conducting',
 'the',
 'routine',
 'sampling',
 'and',
 'analysis',
 'of',
 'water',
 'systems',
 'interpreting',
 'results',
 'maintenance',
 'and',
 'the',
 'installation',


In [17]:
# Step 3, lowercase

step3_tokenized=[]
for description_tokens in description_tokenized:
    step3_tokenized.append([token.lower() for token in description_tokens])


In [18]:
# Step 4, remove length less than 2

step4_tokenized=[]

for description_tokens in step3_tokenized:
    valid_tokens=[]
    for token in description_tokens:
        if len(token) >= 2:
            valid_tokens.append(token)
    step4_tokenized.append(valid_tokens)


In [19]:
# Step 5, remove stopwords using stop words list

stopped_words = []
with open('./stopwords_en.txt') as f:
    stopped_words = f.read().splitlines()
    
step5_tokenized=[]

for description_tokens in step4_tokenized:
    valid_tokens=[]
    for token in description_tokens:
        if token not in stopped_words:
            valid_tokens.append(token)
    step5_tokenized.append(valid_tokens)

In [20]:
# Step 6, getting word list from step5 result

words = list(chain.from_iterable(step5_tokenized))

# Acquire term frequency

term_fd = FreqDist(words)

In [21]:
# Inspect term frequency result

term_fd

FreqDist({'experience': 104062, 'role': 66536, 'work': 64587, 'team': 62308, 'business': 59184, 'skills': 57872, 'working': 52280, 'job': 50814, 'client': 45209, 'sales': 43108, ...})

In [22]:
# number of hapaxes, which apperas only once in document collection

len(term_fd.hapaxes())

48964

In [23]:
# assign a new variable name

appear_once_list = set(term_fd.hapaxes())

In [24]:
# define a function to remove less frequent word

def removeLessFreqWords(article):
    return [w for w in article if w not in appear_once_list]

step6_tokenized = [removeLessFreqWords(article) for article in step5_tokenized]

In [25]:
# Step 7, Document frequency, remove top 50 most frequent words based on document frequency

words_2 = list(chain.from_iterable([set(job) for job in step6_tokenized]))
doc_fd = FreqDist(words_2)
doc_fd.most_common(50)


document_frequency_remove_list=[x[0] for x in doc_fd.most_common(50)]

In [26]:
# Inspect top 50 common document frequency vector

doc_fd.most_common(50)

[('experience', 43644),
 ('role', 34680),
 ('work', 33684),
 ('team', 32585),
 ('working', 30714),
 ('skills', 30412),
 ('client', 26899),
 ('job', 25552),
 ('business', 24739),
 ('uk', 24133),
 ('excellent', 22982),
 ('opportunity', 22678),
 ('company', 22263),
 ('management', 20620),
 ('required', 20555),
 ('development', 20223),
 ('apply', 20133),
 ('based', 19333),
 ('successful', 19118),
 ('join', 18682),
 ('www', 18421),
 ('salary', 18402),
 ('cv', 18383),
 ('support', 18286),
 ('knowledge', 17844),
 ('strong', 16475),
 ('environment', 16408),
 ('posted', 16398),
 ('jobseeking', 16342),
 ('candidate', 16304),
 ('originally', 16294),
 ('leading', 16194),
 ('high', 15922),
 ('service', 15623),
 ('manager', 15587),
 ('good', 15252),
 ('ability', 15154),
 ('including', 14857),
 ('position', 14564),
 ('services', 14501),
 ('benefits', 14434),
 ('training', 14218),
 ('essential', 13915),
 ('experienced', 13826),
 ('key', 13567),
 ('contact', 13551),
 ('level', 13523),
 ('recruitment', 

In [27]:
# remove top 50 most frequent words based on document frequency

def removeCommonFreqWords(article):
    return [w for w in article if w not in document_frequency_remove_list]

step7_tokenized = [removeCommonFreqWords(article) for article in step6_tokenized]

In [28]:
# Generate inspect top 10 2-gram 

words = list(chain.from_iterable(step7_tokenized))
bigrams = ngrams(words, n = 2)
fdbigram = FreqDist(bigrams)

top2gram = fdbigram.most_common(10)
top2gram

[(('employment', 'agency'), 8055),
 (('track', 'record'), 5472),
 (('acting', 'employment'), 5095),
 (('sql', 'server'), 4804),
 (('asp', 'net'), 4687),
 (('relation', 'vacancy'), 3977),
 (('sales', 'executive'), 3619),
 (('chef', 'de'), 3586),
 (('nursing', 'home'), 3503),
 (('de', 'partie'), 3396)]

In [29]:
# Generate vocabulary list in alphabetical order

vocabulary = []

for voc in set(words):
    vocabulary.append(voc)
vocabulary.sort()

In [30]:
# Acquire some statistical information regarding the tokens

def stats_print(tk_reviews):
    words = list(chain.from_iterable(tk_reviews)) # we put all the tokens in the corpus in a single list
    vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words
    lexical_diversity = len(vocab)/len(words)
    print("Vocabulary size: ",len(vocab))
    print("Total number of tokens: ", len(words))
    print("Lexical diversity: ", lexical_diversity)
    print("Total number of reviews:", len(tk_reviews))
    lens = [len(article) for article in tk_reviews]
    print("Average review length:", np.mean(lens))
    print("Maximun review length:", np.max(lens))
    print("Minimun review length:", np.min(lens))
    print("Standard deviation of review length:", np.std(lens))
    
stats_print(step7_tokenized)

Vocabulary size:  40038
Total number of tokens:  6239169
Lexical diversity:  0.0064172007522155594
Total number of reviews: 55449
Average review length: 112.52085700373316
Maximun review length: 990
Minimun review length: 4
Standard deviation of review length: 61.88637513583753


## Saving required outputs
Save the vocabulary, bigrams and job advertisment txt as per spectification.
- vocab.txt
- bigram.txt
- job_ads.txt

In [31]:
# Write bigram.txt from top2bigram list

with open('bigram.txt', 'w') as f:
    for bigram_fre in top2gram:
        f.write(bigram_fre[0][0]+' '+bigram_fre[0][1]+','+str(bigram_fre[1]))
        f.write('\n')

In [32]:
# Write vocab.txt from vocabulary list

with open('vocab.txt', 'w') as f:
    for idx, voc in enumerate(vocabulary):
        f.write(voc+':'+str(idx))
        f.write('\n')

In [33]:
# Write job_ads.txt from all relevant dictionary & list

processed_description =step7_tokenized

with open('job_ads.txt', 'w') as f:
    for idx, job in enumerate(job_list):
        f.write('ID: '+job['Filename'])
        f.write('\n')
        f.write('Category: '+job['Category'])
        f.write('\n')
        f.write('Webindex: '+job['Webindex'])
        f.write('\n')
        f.write('Title: '+job['Title'])
        f.write('\n')
        f.write('Description: '+ ' '.join(processed_description[idx]))
        f.write('\n')

## Summary

Pre-processing text data is an essential step for text mining task. Removing those irrelevant tokens could make the information provided in data more usefull. 

Determining the steps of pre-processing is the hardest part in this part, although the steps have been given in this asssignment. However, it shows a general pre-processing pipline, including extracting text partern, tokenizing text, removing less frequent words, removing most frequent document words, etc.

Also, choosing proper data structure to store data can make the processing steps easier. 

In conclusion, pre-processing requires strong attention details to make sure each step correct.